# VGGNet_D or VGG16

VGG_D is commonly known as VGG16 is considered to be a "very deep network" due to having 16 trainable layer.

VGGNet uses small kernel sizes (uses 3 except in VGGNet_C's 7th, 10th and 13th convolution layers) and stride 1. 

More information can be found in [VERY DEEP CONVOLUTIONAL NETWORKS
FOR LARGE-SCALE IMAGE RECOGNITION](https://arxiv.org/pdf/1409.1556.pdf) by Karen Simonyan& Andrew Zisserman.

### Architecture
input -> conv3-64 -> conv3-64 -> maxpool -> conv3-128 -> conv3-128 -> maxpool -> conv3-256 -> conv3-256 -> conv3-256 -> maxpool -> conv3-512 -> conv3-512 -> conv3-512 -> maxpool -> conv3-512 -> conv3-512 -> conv3-512 -> maxpool -> flatten -> fc4096 -> fc4096 -> output

### Note :
Because of the small image sizes, paddings are changed to "SAME".

In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Input
from keras.layers import Activation
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
from data import get_data_set

In [3]:
# Args
save_dir = "./results"
batch_size = 64
epochs = 100
lr=0.001
decay=0.0005
momentum=0.9
log_dir = "./tensorboard/VGGNet_D_9x9_12/"
# originals
# save_dir = "./results"
# batch_size = 50
# epochs = 50
# lr=0.01
# decay=0.0005
# momentum=0.9

In [4]:
X_train,Y_train = get_data_set("train",input_path = "../input/data_9x9_12band/",one_hot = True)
X_test,Y_test = get_data_set("test",input_path = "../input/data_9x9_12band/", one_hot = True)

X_train.shape

(404, 9, 9, 12)

In [5]:
tensorboard = TensorBoard(log_dir=log_dir,batch_size=batch_size)

In [6]:
inputs = Input(X_train.shape[1:])
inputs

<tf.Tensor 'input_1:0' shape=(?, 9, 9, 12) dtype=float32>

In [7]:
conv_1 = Convolution2D(64, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_1')(inputs)
conv_2 = Convolution2D(64, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_2')(conv_1)
conv_2 = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(conv_2)

In [8]:
conv_3 = Convolution2D(128, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_3')(conv_2)
conv_4 = Convolution2D(128, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_4')(conv_3)
conv_4 = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(conv_4)

In [9]:
conv_5 = Convolution2D(256, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_5')(conv_4)
conv_6 = Convolution2D(256, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_6')(conv_5)
conv_7 = Convolution2D(256, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_7')(conv_6)
conv_7 = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(conv_7)

In [10]:
conv_8 = Convolution2D(512, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_8')(conv_7)
conv_9 = Convolution2D(512, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_9')(conv_8)
conv_10 = Convolution2D(512, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_10')(conv_9)
conv_10 = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(conv_10)

In [11]:
conv_11 = Convolution2D(512, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_11')(conv_10)
conv_12 = Convolution2D(512, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_12')(conv_11)
conv_13 = Convolution2D(512, kernel_size =(3, 3), strides=(1, 1), padding='same', activation='relu',
                           name='conv_13')(conv_12)
conv_13 = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(conv_13)

In [12]:
dense_1 = Flatten(name='flatten')(conv_13)

In [13]:
dense_1 = Dense(4096, activation='relu', name='dense_1')(dense_1)
dense_2 = Dropout(0.5)(dense_1)
dense_2 = Dense(4096, activation='relu', name='dense_2')(dense_2)
dense_3 = Dropout(0.5)(dense_2)
dense_3 = Dense(Y_train.shape[1], name='dense_3')(dense_3)
prediction = Activation('softmax', name='softmax')(dense_3)
prediction

<tf.Tensor 'softmax/Softmax:0' shape=(?, 4) dtype=float32>

In [14]:
model = Model(outputs=prediction,inputs = inputs)

# Lets train

In [15]:
# optimizer
sgd = SGD(lr=lr, decay=decay, momentum=momentum)

In [16]:
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 9, 9, 12)          0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 9, 9, 64)          6976      
_________________________________________________________________
conv_2 (Conv2D)              (None, 9, 9, 64)          36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 5, 5, 128)         73856     
_________________________________________________________________
conv_4 (Conv2D)              (None, 5, 5, 128)         147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0         
__________

In [18]:
model.fit(x = X_train, y = Y_train, batch_size=batch_size, epochs=epochs,
              validation_data=[X_test,Y_test],shuffle = True,
              callbacks=[tensorboard])

Train on 404 samples, validate on 793 samples
Epoch 1/100
404/404 [==============================] - 16s 40ms/step - loss: 1.3877 - acc: 0.2847 - val_loss: 1.3871 - val_acc: 0.3922
Epoch 2/100
404/404 [==============================] - 15s 38ms/step - loss: 1.3850 - acc: 0.3020 - val_loss: 1.3793 - val_acc: 0.2535
Epoch 3/100
404/404 [==============================] - 15s 36ms/step - loss: 1.3801 - acc: 0.3094 - val_loss: 1.3703 - val_acc: 0.2799
Epoch 4/100
404/404 [==============================] - 15s 37ms/step - loss: 1.3759 - acc: 0.3490 - val_loss: 1.3611 - val_acc: 0.4691
Epoch 5/100
404/404 [==============================] - 13s 33ms/step - loss: 1.3725 - acc: 0.3465 - val_loss: 1.3528 - val_acc: 0.5624
Epoch 6/100
404/404 [==============================] - 16s 40ms/step - loss: 1.3680 - acc: 0.3614 - val_loss: 1.3421 - val_acc: 0.5460
Epoch 7/100
404/404 [==============================] - 15s 36ms/step - loss: 1.3605 - acc: 0.3564 - val_loss: 1.3278 - val_acc: 0.3934
Epoch 8/1

Epoch 61/100
404/404 [==============================] - 18s 43ms/step - loss: 0.0554 - acc: 0.9777 - val_loss: 0.6739 - val_acc: 0.8184
Epoch 62/100
404/404 [==============================] - 20s 49ms/step - loss: 0.0771 - acc: 0.9728 - val_loss: 0.6689 - val_acc: 0.8272
Epoch 63/100
404/404 [==============================] - 17s 41ms/step - loss: 0.0531 - acc: 0.9752 - val_loss: 0.5682 - val_acc: 0.8424
Epoch 64/100
404/404 [==============================] - 20s 49ms/step - loss: 0.0644 - acc: 0.9777 - val_loss: 0.5892 - val_acc: 0.8474
Epoch 65/100
404/404 [==============================] - 17s 41ms/step - loss: 0.0703 - acc: 0.9827 - val_loss: 0.5203 - val_acc: 0.8676
Epoch 66/100
404/404 [==============================] - 20s 49ms/step - loss: 0.0466 - acc: 0.9777 - val_loss: 0.5907 - val_acc: 0.8512
Epoch 67/100
404/404 [==============================] - 17s 42ms/step - loss: 0.0911 - acc: 0.9703 - val_loss: 0.6244 - val_acc: 0.8462
Epoch 68/100
404/404 [==========================